# [Paris Saclay Center for Data Science](http://www.datascience-paris-saclay.fr)


## [Kaggle Seguro RAMP](http://www.ramp.studio/problems/kaggle_seguro): Kaggle Porto-Seguro safe driver prediction

_Balázs Kégl (LAL/CNRS)_

## Introduction
This is a [Kaggle data challenge](https://www.kaggle.com/c/porto-seguro-safe-driver-prediction) on predicting the probability that a driver will initiate an auto insurance claim in the next year.

### Requirements

* numpy>=1.10.0  
* matplotlib>=1.5.0 
* pandas>=0.19.0  
* scikit-learn>=0.19   

In [1]:
%matplotlib inline
import os
import glob
import numpy as np
from scipy import io
import matplotlib.pyplot as plt
import pandas as pd

## Exploratory data analysis

### Loading the data

The repo contains mock data in `/data`, simulating the format of the official Kaggle data, but smaller in size and containing random features. If you want to execute the notebook on the official Kaggle data, sign up to the [challenge](https://www.kaggle.com/c/porto-seguro-safe-driver-prediction), download `train.7z` and `test.7z`, unzip them and place them in `kaggle_data/`. If you want to use the starting kit to generate output in the right Kaggle submission format, you will also need to download `sample_submission.7z`, unzip it, and place it in `kaggle_data/`.

In [8]:
train_filename = 'kaggle_data/train.csv'

In [17]:
train_filename = 'kaggle_data/test.csv'

In [18]:
#train_filename = 'data/train.csv'

In [19]:
data = pd.read_csv(train_filename)

In [20]:
data.head()

,id,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,0,0,1,8,1,0,0,1,0,0,...,1,1,1,12,0,1,1,0,0,1
1,1,4,2,5,1,0,0,0,0,1,...,2,0,3,10,0,0,1,1,0,1
2,2,5,1,3,0,0,0,0,0,1,...,4,0,2,4,0,0,0,0,0,0
3,3,0,1,6,0,0,1,0,0,0,...,5,1,0,5,1,0,1,0,0,0
4,4,5,1,7,0,0,0,0,0,1,...,4,0,0,4,0,1,1,0,0,1


In [21]:
data.describe()

,id,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
count,8.928160e+05,892816.000000,892816.000000,892816.000000,892816.000000,892816.000000,892816.000000,892816.000000,892816.000000,892816.000000,...,892816.000000,892816.000000,892816.000000,892816.000000,892816.000000,892816.000000,892816.000000,892816.000000,892816.000000,892816.000000
mean,7.441535e+05,1.902371,1.358613,4.413734,0.417361,0.408132,0.393246,0.257191,0.163659,0.185905,...,5.438478,1.440265,2.875013,7.540367,0.123720,0.627756,0.554660,0.287796,0.349344,0.152428
std,4.296830e+05,1.986503,0.663002,2.700149,0.493453,1.355068,0.488471,0.437086,0.369966,0.389030,...,2.330081,1.200620,1.694072,2.745882,0.329262,0.483403,0.497004,0.452736,0.476763,0.359435
min,0.000000e+00,0.000000,-1.000000,0.000000,-1.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.720218e+05,0.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,4.000000,1.000000,2.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,7.443070e+05,1.000000,1.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,5.000000,1.000000,3.000000,7.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000
75%,1.116308e+06,3.000000,2.000000,6.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,...,7.000000,2.000000,4.000000,9.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000
max,1.488026e+06,7.000000,4.000000,11.000000,1.000000,6.000000,1.000000,1.000000,1.000000,1.000000,...,20.000000,11.000000,15.000000,28.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [22]:
data.dtypes

id                  int64
ps_ind_01           int64
ps_ind_02_cat       int64
ps_ind_03           int64
ps_ind_04_cat       int64
ps_ind_05_cat       int64
ps_ind_06_bin       int64
ps_ind_07_bin       int64
ps_ind_08_bin       int64
ps_ind_09_bin       int64
ps_ind_10_bin       int64
ps_ind_11_bin       int64
ps_ind_12_bin       int64
ps_ind_13_bin       int64
ps_ind_14           int64
ps_ind_15           int64
ps_ind_16_bin       int64
ps_ind_17_bin       int64
ps_ind_18_bin       int64
ps_reg_01         float64
ps_reg_02         float64
ps_reg_03         float64
ps_car_01_cat       int64
ps_car_02_cat       int64
ps_car_03_cat       int64
ps_car_04_cat       int64
ps_car_05_cat       int64
ps_car_06_cat       int64
ps_car_07_cat       int64
ps_car_08_cat       int64
ps_car_09_cat       int64
ps_car_10_cat       int64
ps_car_11_cat       int64
ps_car_11           int64
ps_car_12         float64
ps_car_13         float64
ps_car_14         float64
ps_car_15         float64
ps_calc_01  

In [23]:
data.count()

id                892816
ps_ind_01         892816
ps_ind_02_cat     892816
ps_ind_03         892816
ps_ind_04_cat     892816
ps_ind_05_cat     892816
ps_ind_06_bin     892816
ps_ind_07_bin     892816
ps_ind_08_bin     892816
ps_ind_09_bin     892816
ps_ind_10_bin     892816
ps_ind_11_bin     892816
ps_ind_12_bin     892816
ps_ind_13_bin     892816
ps_ind_14         892816
ps_ind_15         892816
ps_ind_16_bin     892816
ps_ind_17_bin     892816
ps_ind_18_bin     892816
ps_reg_01         892816
ps_reg_02         892816
ps_reg_03         892816
ps_car_01_cat     892816
ps_car_02_cat     892816
ps_car_03_cat     892816
ps_car_04_cat     892816
ps_car_05_cat     892816
ps_car_06_cat     892816
ps_car_07_cat     892816
ps_car_08_cat     892816
ps_car_09_cat     892816
ps_car_10_cat     892816
ps_car_11_cat     892816
ps_car_11         892816
ps_car_12         892816
ps_car_13         892816
ps_car_14         892816
ps_car_15         892816
ps_calc_01        892816
ps_calc_02        892816


In [24]:
np.unique(data['target'])

KeyError: 'target'

In [ ]:
data.groupby('target').count()[['id']]

## The pipeline

For submitting at the [RAMP site](http://ramp.studio), you will have to write two classes, saved in two different files,
* the class `FeatureExtractor`, which will be used to extract features for classification from the dataset and produce a numpy array of size (number of samples $\times$ number of features), and  
* the class `Classifier` to predict the target.

### Feature extractor

The feature extractor implements a `transform` member function. It is saved in the file [`submissions/starting_kit/feature_extractor.py`](/edit/submissions/starting_kit/feature_extractor.py). It receives the pandas dataframe `X_df` defined at the beginning of the notebook. It should produce a numpy array representing the extracted features, which will then be used for the classification.  

Note that the following code cells are *not* executed in the notebook. The notebook saves their contents in the file specified in the first line of the cell, so you can edit your submission before running the local test below and submitting it at the RAMP site.

In [10]:
%%file submissions/starting_kit/feature_extractor.py
class FeatureExtractor():
    def __init__(self):
        pass

    def fit(self, X_df, y):
        pass

    def transform(self, X_df):
        return X_df.values



Overwriting submissions/starting_kit/feature_extractor.py


### Classifier

The classifier follows a classical scikit-learn classifier template. It should be saved in the file [`submissions/starting_kit/classifier.py`](/edit/submissions/starting_kit/classifier.py). In its simplest form it takes a scikit-learn pipeline, assigns it to `self.clf` in `__init__`, then calls its `fit` and `predict_proba` functions in the corresponding member funtions.

In [11]:
%%file submissions/starting_kit/classifier.py
from sklearn.base import BaseEstimator
from sklearn.ensemble import RandomForestClassifier


class Classifier(BaseEstimator):
    def __init__(self):
        pass

    def fit(self, X, y):
        self.clf = RandomForestClassifier(
            n_estimators=2, max_leaf_nodes=2, random_state=61)
        self.clf.fit(X, y)

    def predict(self, X):
        return self.clf.predict(X)

    def predict_proba(self, X):
        return self.clf.predict_proba(X)



Overwriting submissions/starting_kit/classifier.py


## Local testing (before submission)

It is <b><span style="color:red">important that you test your submission files before submitting them</span></b>. For this we provide a unit test. Note that the test runs on your files in [`submissions/starting_kit`](/tree/submissions/starting_kit), not on the classes defined in the cells of this notebook.

First `pip install ramp-workflow` or install it from the [github repo](https://github.com/paris-saclay-cds/ramp-workflow). Make sure that the python files `classifier.py` and `feature_extractor.py` are in the  [`submissions/starting_kit`](/tree/submissions/starting_kit) folder, and the data `train.csv` and `test.csv` are in [`data`](/tree/data). Then run

```ramp_test_submission```

If it runs and print training and test errors on each fold, then you can submit the code.

Note that `kaggle_data/test.csv` is the actual Kaggle test file, so we have no test labels. To not to crash the test, we mock all 0 labels for the test points. This means that the **test scores are not meaningful** (only he valid scores are).

In [12]:
!ramp_test_submission

Testing Kaggle Porto-Seguro safe driver prediction
Reading train and test files from ./data ...
Traceback (most recent call last):
  File "/anaconda/envs/python36/bin/ramp_test_submission", line 11, in <module>
    load_entry_point('ramp-workflow==0.1.0+19.g6ba5470', 'console_scripts', 'ramp_test_submission')()
  File "/anaconda/envs/python36/lib/python3.6/site-packages/rampwf/utils/command_line.py", line 85, in ramp_test_submission
    retrain=retrain)
  File "/anaconda/envs/python36/lib/python3.6/site-packages/rampwf/utils/testing.py", line 186, in assert_submission
    X_train, y_train, X_test, y_test = assert_data(ramp_kit_dir, ramp_data_dir)
  File "/anaconda/envs/python36/lib/python3.6/site-packages/rampwf/utils/testing.py", line 86, in assert_data
    X_train, y_train = problem.get_train_data(path=ramp_data_dir)
  File "./problem.py", line 34, in get_train_data
    data = pd.read_csv(os.path.join(path, 'kaggle_data', 'train.csv'))
  File "/anaconda/envs/python36/lib/python3.6/si

You can use the `--quick-test` switch to test the notebook on the mock data sets in `data/`. Since the data is random, the scores will not be meaningful, but it can be useful to run this first on your submissions to make sure they run without errors.

In [13]:
!ramp_test_submission --quick-test

Testing Kaggle Porto-Seguro safe driver prediction
Reading train and test files from ./data ...
Reading cv ...
Training ./submissions/starting_kit ...
CV fold 0
	train ngini = 0.175
	valid ngini = 0.109
	test ngini = 0.879
	train auc = 0.649
	valid auc = 0.509
	test auc = 0.47
	train acc = 0.965
	valid acc = 0.965
	test acc = 0.99
	train nll = 0.171
	valid nll = 0.201
	test nll = 0.066
CV fold 1
	train ngini = 0.251
	valid ngini = 0.155
	test ngini = 0.879
	train auc = 0.663
	valid auc = 0.486
	test auc = 0.47
	train acc = 0.965
	valid acc = 0.965
	test acc = 0.99
	train nll = 0.146
	valid nll = 0.164
	test nll = 0.059
CV fold 2
	train ngini = 0.395
	valid ngini = 0.021
	test ngini = 0.798
	train auc = 0.717
	valid auc = 0.543
	test auc = 0.449
	train acc = 0.965
	valid acc = 0.965
	test acc = 0.99
	train nll = 0.137
	valid nll = 0.156
	test nll = 0.069
CV fold 3
	train ngini = 0.322
	valid ngini = 0.36
	test ngini = 0.455
	train auc = 0.645
	valid auc = 0.654
	test auc = 0.364
	train 

## Other models in the starting kit

You can also keep several other submissions in your work directory [`submissions`](/tree/submissions) and test them using
```
ramp_test_submission --submission <submission_name>
```
where `<submission_name>` is the name of the folder in `submissions/`.

## Submitting to Kaggle

You can use this starting kit to train models and submit their predictions to Kaggle. `problem.save_y_pred` implements outputting the predictions. You can turn on this using the `--save-y-preds` switch:
```
ramp_test_submission --submission <submission_name> --save-y-preds
```
This will create the arborescence
```
submissions/<submission_name>/training_output
├── bagged_test_scores.csv
├── bagged_train_valid_scores.csv
├── fold_0
│   └── y_pred_test.csv
├── ...
├── fold_<k-1>
│   └── y_pred_test.csv
└── y_pred_bagged_test.csv
```
You can find test prediction vectors in each fold folder `submissions/<submission_name>/training_output/fold_<i>` and the bagged prediction vector **`submissions/<submission_name>/training_output/y_pred_bagged_test.csv`**. It is this latter that you should submit to Kaggle.

If your goal is to use this starting kit to optimize your Kaggle submission, besides optimizing your feature extractor and classifier, you can also tune the CV bagging scheme by changing the type of cross validation, the number of folds, and the test proportion in `problem.get_cv`. We found that `test_size=0.5` worked well with an extreme large number of folds, typically `n_splits=64`, but these parameters depend on the classifier you are testing, so may need fine tuning. 

## Submitting to [ramp.studio](http://ramp.studio)

If you are eligible, you can join the team at [ramp.studio](http://www.ramp.studio). First, if it is your first time using RAMP, [sign up](http://www.ramp.studio/sign_up), otherwise [log in](http://www.ramp.studio/login). Then ask for a sign-up to the event [kaggle_seguro](http://www.ramp.studio/events/kaggle_seguro). Both signups are controled by RAMP administrators, so there **can be a delay between asking for signup and being able to submit**.

Once your signup request is accepted, you can go to your [sandbox](http://www.ramp.studio/events/kaggle_seguro/sandbox) and copy-paste (or upload) [`feature_extractor.py`](/edit/submissions/starting_kit/feature_extractor.py) and [`classifier.py`](/edit/submissions/starting_kit/classifier.py) from `submissions/starting_kit`. Save it, rename it, then submit it. The submission is trained and tested on our backend in the same way as `ramp_test_submission` does it locally. While your submission is waiting in the queue and being trained, you can find it in the "New submissions (pending training)" table in [my submissions](http://www.ramp.studio/events/kaggle_seguro/my_submissions). Once it is trained, you get a mail, and your submission shows up on the [public leaderboard](http://www.ramp.studio/events/kaggle_seguro/leaderboard). 
If there is an error (despite having tested your submission locally with `ramp_test_submission`), it will show up in the "Failed submissions" table in [my submissions](http://www.ramp.studio/events/kaggle_seguro/my_submissions). You can click on the error to see part of the trace.

After submission, do not forget to give credits to the previous submissions you reused or integrated into your submission.

The data set we use at the backend is usually different from what you find in the starting kit, so the score may be different.

The usual way to work with RAMP is to explore solutions, add feature transformations, select models, perhaps do some AutoML/hyperopt, etc., _locally_, and checking them with `ramp_test_submission`. The script prints mean cross-validation and test scores 
```
----------------------------
train ngini = 0.119 ± 0.007
train auc = 0.559 ± 0.003
train acc = 0.964 ± 0.0
train nll = 0.156 ± 0.0
valid ngini = 0.114 ± 0.005
valid auc = 0.558 ± 0.002
valid acc = 0.964 ± 0.0
valid nll = 0.156 ± 0.0
test ngini = 0.229 ± 0.256
test auc = 0.307 ± 0.064
test acc = 1.0 ± 0.0
test nll = 0.037 ± 0.0
```
and bagged cross-validation and test scores
```
valid ngini = 0.167
test ngini = -0.324
```
This latter combines the cross-validation models pointwise on the validation and test sets, and usually leads to a better score than the mean CV score. The RAMP [leaderboard](http://www.ramp.studio/events/kaggle_seguro/leaderboard) displays this score.

The official score in this RAMP (the first score column after "historical contributivity" on the [leaderboard](http://www.ramp.studio/events/kaggle_seguro/leaderboard)) is normalized Gini ("ngini"), so the line that is relevant in the output of `ramp_test_submission` is `valid ngini = 0.167`. When the score is good enough, you can submit it at the RAMP.

## More information

You can find more information in the [README](https://github.com/paris-saclay-cds/ramp-workflow/blob/master/README.md) of the [ramp-workflow library](https://github.com/paris-saclay-cds/ramp-workflow).

## Contact

Don't hesitate to [contact us](mailto:admin@ramp.studio?subject=kaggle seguro notebook).